<a href="https://colab.research.google.com/github/Shaitender/Projects-poc/blob/master/Cement_Strength_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regression Model in Keras

<a id="item31"></a>

### Download and Clean Dataset

In [0]:
import pandas as pd
import numpy as np

In [0]:
concrete_data = pd.read_csv('https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [0]:
#### Let's check how many data points we have.
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


#### Let's check the dataset for any missing values.

In [0]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [0]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>

In [0]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [0]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [0]:
target.shape

(1030,)

In [0]:
predictors.shape

(1030, 8)

##  Normalize the data by substracting the mean and dividing by the standard deviation.

In [0]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [0]:
predictors_norm.shape

(1030, 8)

Let's save the number of predictors to *n_cols* since we will need this number when building our network.

In [0]:
n_cols = predictors_norm.shape[1] # number of predictors

In [0]:
n_cols

8

##  Import the Keras Library

In [0]:
import keras

Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item34'></a>

## Build a Regression  Neural Network

In [0]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

## A. Train and Test the Network

#### train and evaluation 50 times, i.e., create a list of 50 mean squared errors

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [0]:
mse_list = []
mse_list1 = []
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3)
    model = regression_model()
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50)
    mse_list1.append(model.evaluate(x=X_test, y=y_test))
    mse_list.append(mean_squared_error(y_test, model.predict(X_test)))
    print('\n ++++++++++++  End of loop number -> {0} ++++++++++++++++++ \n \n'.format(i+1))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 721 samples, validate on 309 samples
Epoch 1/50
721/721 [==============================] - 2s 2ms/step - loss: 15293.8095 - val_loss: 5120.4856
Epoch 2/50
721/721 [==============================] - 0s 322us/step - loss: 3006.6170 - val_loss: 1697.8761
Epoch 3/50
721/721 [==============================] - 0s 291us/step - loss: 1728.6721 - val_loss: 1728.4266
Epoch 4/50
721/721 [==============================] - 0s 343us/step - loss: 1613.9825 - val_loss: 1556.4852
Epoch 5/50
721/721 [==============================] - 0s 396us/step - loss: 1530.6234 - val_loss: 1479.6981
Epoch 6/50
721/721 [==============================] - 0s 365us/step - loss: 1451.0646 - val_loss: 1405.5126
Epoch 7/50
721/721 [==============================] - 0s 333us/step - loss: 1371.8353 - val_loss: 1336.8779
Epoch 8/50
721/721 [==============================] - 0s 310us/step - loss: 129

In [0]:
print ('Mean of Mean Square Error ', np.mean(mse_list))
print ('Standard Deviation of MSE', np.std(mse_list))

Mean of Mean Square Error  368.2300057437543
Standard Deviation of MSE 432.7098626977509


### Notes ==> Section A 
####  MSE on test dataset -> 368.23 and the standard deviation is -> 432.70

<a id='item36'></a>

## B. Train and test with normalized data

In [0]:
mse_list = []
mse_list_1 = []
for i in range(50):
    X_train_norm, X_test_norm, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3)
    model = regression_model()
    model.fit(X_train_norm, y_train, validation_data=(X_test_norm, y_test), epochs=50)
    mse_list.append(model.evaluate(x=X_test_norm, y=y_test))
    mse_list_1.append(mean_squared_error(y_test, model.predict(X_test_norm)))
    print('\n ++++++++++++  End of loop number -> {0} ++++++++++++++++++ \n \n'.format(i+1))

Train on 721 samples, validate on 309 samples
Epoch 1/50
721/721 [==============================] - 8s 12ms/step - loss: 1570.5314 - val_loss: 1411.9227
Epoch 2/50
721/721 [==============================] - 0s 448us/step - loss: 1552.4205 - val_loss: 1395.0361
Epoch 3/50
721/721 [==============================] - 0s 446us/step - loss: 1533.7348 - val_loss: 1377.5237
Epoch 4/50
721/721 [==============================] - 0s 404us/step - loss: 1514.3849 - val_loss: 1359.2572
Epoch 5/50
721/721 [==============================] - 0s 520us/step - loss: 1494.1840 - val_loss: 1340.7884
Epoch 6/50
721/721 [==============================] - 0s 498us/step - loss: 1473.4312 - val_loss: 1321.0541
Epoch 7/50
721/721 [==============================] - 0s 504us/step - loss: 1451.5150 - val_loss: 1300.8121
Epoch 8/50
721/721 [==============================] - 0s 434us/step - loss: 1428.7527 - val_loss: 1279.6948
Epoch 9/50
721/721 [==============================] - 0s 267us/step - loss: 1404.9179 - val

In [0]:
print ('Mean of MSE = %.2f' % np.mean(mse_list_1))
print ('Standard Deviation of MSE = %.2f' % np.std(mse_list_1))

Mean of MSE = 350.31
Standard Deviation of MSE = 79.23


### Notes ==> Section B 

#### By using the normalized training dataset  the mean MSE of the test data is increased to 350.31 (from 368.23). Also the standard deviation is reduced to 79.23 (from 432.70). 


## C. Increase the number of epochs

In [0]:
mse_list = []
mse_list_2 = []
for i in range(50):
    X_train_norm, X_test_norm, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3)
    model = regression_model()
    model.fit(X_train_norm, y_train, validation_data=(X_test_norm, y_test), epochs=100)
    mse_list.append(model.evaluate(x=X_test_norm, y=y_test))
    mse_list_2.append(mean_squared_error(y_test, model.predict(X_test_norm)))
    print('\n ++++++++++++  End of loop number -> {0} ++++++++++++++++++ \n \n'.format(i+1))

Train on 721 samples, validate on 309 samples
Epoch 1/100
721/721 [==============================] - 6s 8ms/step - loss: 1573.6501 - val_loss: 1567.8208
Epoch 2/100
721/721 [==============================] - 0s 268us/step - loss: 1557.8372 - val_loss: 1553.1355
Epoch 3/100
721/721 [==============================] - 0s 249us/step - loss: 1542.2949 - val_loss: 1538.7709
Epoch 4/100
721/721 [==============================] - 0s 253us/step - loss: 1526.9804 - val_loss: 1523.9719
Epoch 5/100
721/721 [==============================] - 0s 237us/step - loss: 1511.4526 - val_loss: 1509.1826
Epoch 6/100
721/721 [==============================] - 0s 264us/step - loss: 1495.6748 - val_loss: 1494.2017
Epoch 7/100
721/721 [==============================] - 0s 256us/step - loss: 1479.8098 - val_loss: 1478.3842
Epoch 8/100
721/721 [==============================] - 0s 331us/step - loss: 1463.0728 - val_loss: 1461.9764
Epoch 9/100
721/721 [==============================] - 0s 266us/step - loss: 1445.64

In [0]:
print ('Mean of MSE 2 = %.2f' % np.mean(mse_list_2))
print ('Standard Deviation of MSE 2 = %.2f' % np.std(mse_list_2))

Mean of MSE 2 = 170.94
Standard Deviation of MSE 2 = 24.01


### Notes ==> Section C
#### By increasing the number of training epochs, the mean MSE of the test data is further improved to 170.94 (from  350.31). Also the standard deviation is reduced to 24.01 (from  79.23) which is better.

## D. Increase the number of hidden layers

In [0]:
# define regression model
def regression_model_extra_layers():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [0]:
mse_list = []
mse_list_3 = []
for i in range(50):
    X_train_norm, X_test_norm, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3)
    model_1 = regression_model_extra_layers()
    model_1.fit(X_train_norm, y_train, validation_data=(X_test_norm, y_test), epochs=50)
    mse_list.append(model_1.evaluate(x=X_test_norm, y=y_test))
    mse_list_3.append(mean_squared_error(y_test, model_1.predict(X_test_norm)))
    print('\n ++++++++++++  End of loop number -> {0} ++++++++++++++++++ \n \n'.format(i+1))

Train on 721 samples, validate on 309 samples
Epoch 1/50
721/721 [==============================] - 22s 30ms/step - loss: 1578.5452 - val_loss: 1513.6376
Epoch 2/50
721/721 [==============================] - 1s 877us/step - loss: 1551.8383 - val_loss: 1484.3262
Epoch 3/50
721/721 [==============================] - 1s 885us/step - loss: 1516.9323 - val_loss: 1441.9229
Epoch 4/50
721/721 [==============================] - 1s 980us/step - loss: 1462.8971 - val_loss: 1375.4631
Epoch 5/50
721/721 [==============================] - 1s 1ms/step - loss: 1376.4717 - val_loss: 1269.3737
Epoch 6/50
721/721 [==============================] - 1s 855us/step - loss: 1244.2712 - val_loss: 1108.6094
Epoch 7/50
721/721 [==============================] - 1s 863us/step - loss: 1052.4599 - val_loss: 891.5354
Epoch 8/50
721/721 [==============================] - 1s 849us/step - loss: 804.4571 - val_loss: 638.4665
Epoch 9/50
721/721 [==============================] - 1s 869us/step - loss: 540.3250 - val_loss

In [0]:
print ('Mean of MSE = %.2f' % np.mean(mse_list_3))
print ('Standard Deviation of MSE = %.2f' % np.std(mse_list_3))

Mean of MSE = 130.36
Standard Deviation of MSE = 14.85


### Notes ==> Section D
#### Increase the number of hidden layers, the mean MSE of the test data is further improved to 130.36 (from  170.94). Also the standard deviation is reduced to 14.85 (from 24.01) which is best in all section.